# Machine Learning Fairness Algorithms Evaluation

#### Fairness in Machine Learning
The aim of a fairness algorithm is to avoid the outcome decisions are being made unfairly to certain groups or individuals.
- Algorithms are made by humans and trained by data which may be biased
- However, there are many definitions of fairness that cannot be optimized at the same time


#### Learning fair representations (LFR)
The main idea: map each individual, represented as a data point in a given input space, to a probability distribution in a new representation space. The aim of this new representation is to lose any information that can identify whether the person belongs to the protected subgroup, while retaining as much other information as possible.

A discriminative clustering model, where the prototypes act as the clusters

#### Prejudice Remover Regularizer (PR)

ADD DETAILS


### Data Pre-processing

In [29]:
import csv
import pandas as pd

In [30]:
data = '../data/compas-scores.csv'

In [31]:
compas_scores = pd.read_csv(data)
compas_scores.head()

,id,name,first,last,compas_screening_date,sex,dob,age,age_cat,race,...,vr_offense_date,vr_charge_desc,v_type_of_assessment,v_decile_score,v_score_text,v_screening_date,type_of_assessment,decile_score.1,score_text,screening_date
0,1,miguel hernandez,miguel,hernandez,2013-08-14,Male,1947-04-18,69,Greater than 45,Other,...,NaN,NaN,Risk of Violence,1,Low,2013-08-14,Risk of Recidivism,1,Low,2013-08-14
1,2,michael ryan,michael,ryan,2014-12-31,Male,1985-02-06,31,25 - 45,Caucasian,...,NaN,NaN,Risk of Violence,2,Low,2014-12-31,Risk of Recidivism,5,Medium,2014-12-31
2,3,kevon dixon,kevon,dixon,2013-01-27,Male,1982-01-22,34,25 - 45,African-American,...,2013-07-05,Felony Battery (Dom Strang),Risk of Violence,1,Low,2013-01-27,Risk of Recidivism,3,Low,2013-01-27
3,4,ed philo,ed,philo,2013-04-14,Male,1991-05-14,24,Less than 25,African-American,...,NaN,NaN,Risk of Violence,3,Low,2013-04-14,Risk of Recidivism,4,Low,2013-04-14
4,5,marcu brown,marcu,brown,2013-01-13,Male,1993-01-21,23,Less than 25,African-American,...,NaN,NaN,Risk of Violence,6,Medium,2013-01-13,Risk of Recidivism,8,High,2013-01-13


In [36]:
compas_scores.two_year_recid

AttributeError: 'DataFrame' object has no attribute 'two_year_recid'

In [33]:
compas_scores.dtypes

id                           int64
name                        object
first                       object
last                        object
compas_screening_date       object
sex                         object
dob                         object
age                          int64
age_cat                     object
race                        object
juv_fel_count                int64
decile_score                 int64
juv_misd_count               int64
juv_other_count              int64
priors_count                 int64
days_b_screening_arrest    float64
c_jail_in                   object
c_jail_out                  object
c_case_number               object
c_offense_date              object
c_arrest_date               object
c_days_from_compas         float64
c_charge_degree             object
c_charge_desc               object
is_recid                     int64
num_r_cases                float64
r_case_number               object
r_charge_degree             object
r_days_from_arrest  

In [19]:
compas_scores_race = compas_scores.loc[compas_scores['race'].isin(['Caucasian' ,'African-American'])]
compas_scores_race

KeyError: 'race'

In [ ]:
# converting to binary data
df_one = pd.get_dummies(compas_scores_race["race"])
# print(df_one)
 
# display result
df_two = pd.concat((df_one, compas_scores_race), axis=1)
df_two = df_two.drop(["race"], axis=1)
df_two = df_two.drop(["Caucasian"], axis=1)
new_df = df_two.rename(columns={"African-American": "race"})
new_df